# Welcome to Full Stack Machine Learning's Week 4 Project!

In the final week, you will return to the workflow you built last week on the [taxi dataset](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page). 

## Task 1: Deploy the champion
Use what you have learned in the last two weeks to make necessary modifications and to deploy your latest version of the `TaxiFarePrediction` flow to Argo. Use `--branch champion` to denote this deployment as the champion model.

In [6]:
%%writefile ../flows/cloud/event_triggered_linear_regression.py
from metaflow import FlowSpec, step, card, conda_base, current, Parameter, Flow, trigger, project
from metaflow.cards import Markdown, Table, Image, Artifact

URL = "https://outerbounds-datasets.s3.us-west-2.amazonaws.com/taxi/latest.parquet"
DATETIME_FORMAT = "%Y-%m-%d %H:%M:%S"

@project(name="champion")
@trigger(events=["s3"])
@conda_base(
    libraries={
        "pandas": "1.4.2",
        "pyarrow": "11.0.0",
        "numpy": "1.22.4",
        "scikit-learn": "1.1.2",
    }
)
class TaxiFarePrediction(FlowSpec):
    data_url = Parameter("data_url", default=URL)

    def transform_features(self, df):
        # TODO:
        obviously_bad_data_filters = [
                df.fare_amount > 0,  # fare_amount in US Dollars
                df.trip_distance <= 100,  # trip_distance in miles
                df.trip_distance > 0
             ]
        for f in obviously_bad_data_filters:
            df = df[f]
        # Try to complete tasks 2 and 3 with this function doing nothing like it currently is.
        # Understand what is happening.
        # Revisit task 1 and think about what might go in this function.

        return df

    @step
    def start(self):
        import pandas as pd
        from sklearn.model_selection import train_test_split

        self.df = self.transform_features(pd.read_parquet(self.data_url))
        # NOTE: we are split into training and validation set in the validation step which uses cross_val_score.
        # This is a simple/naive way to do this, and is meant to keep this example simple, to focus learning on deploying Metaflow flows.
        # In practice, you want split time series data in more sophisticated ways and run backtests.
        self.X = self.df["trip_distance"].values.reshape(-1, 1)
        self.y = self.df["total_amount"].values
        # self.y = self.df["fare_amount"].values
        self.next(self.linear_model)

    @step
    def linear_model(self):
        "Fit a single variable, linear model to the data."
        from sklearn.linear_model import LinearRegression

        # TODO: Play around with the model if you are feeling it.
        self.model = LinearRegression()
        self.model.fit(self.X,self.y)
        
        self.next(self.validate)

    def gather_sibling_flow_run_results(self):
        # storage to populate and feed to a Table in a Metaflow card
        rows = []

        # loop through runs of this flow
        for run in Flow(self.__class__.__name__):
            if run.id != current.run_id:
                if run.successful:
                    icon = "✅"
                    msg = "OK"
                    score = str(run.data.scores.mean())
                else:
                    icon = "❌"
                    msg = "Error"
                    score = "NA"
                    for step in run:
                        for task in step:
                            if not task.successful:
                                msg = task.stderr
                row = [
                    Markdown(icon),
                    Artifact(run.id),
                    Artifact(run.created_at.strftime(DATETIME_FORMAT)),
                    Artifact(score),
                    Markdown(msg),
                ]
                rows.append(row)
            else:
                rows.append(
                    [
                        Markdown("✅"),
                        Artifact(run.id),
                        Artifact(run.created_at.strftime(DATETIME_FORMAT)),
                        Artifact(str(self.scores.mean())),
                        Markdown("This run..."),
                    ]
                )
        return rows

    @card(type="corise")
    @step
    def validate(self):
        from sklearn.model_selection import cross_val_score

        self.scores = cross_val_score(self.model, self.X, self.y, cv=5)
        current.card.append(Markdown("# Taxi Fare Prediction Results"))
        current.card.append(
            Table(
                self.gather_sibling_flow_run_results(),
                headers=["Pass/fail", "Run ID", "Created At", "R^2 score", "Stderr"],
            )
        )
        self.next(self.end)

    @step
    def end(self):
        print("Success!")


if __name__ == "__main__":
    TaxiFarePrediction()

Overwriting ../flows/cloud/event_triggered_linear_regression.py


In [7]:
! python ../flows/cloud/event_triggered_linear_regression.py --environment=conda --production --branch champion --production argo-workflows create


Metaflow 2.10.6+ob(v1) executing TaxiFarePrediction for user:sandbox
Project: champion, Branch: prod.champion
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Deploying champion.prod.champion.taxifareprediction to Argo Workflows...
It seems this is the first time you are deploying champion.prod.champion.taxifareprediction to Argo Workflows.

A new production token generated.

The namespace of this production flow is
    production:mfprj-fz6nmearex563gd6-0-wtag
To analyze results of this production flow add this line in your notebooks:
    namespace("production:mfprj-fz6nmearex563gd6-0-wtag")
If you want to authorize other people to deploy new versions of this flow to Argo Workflows, they need to call
    argo-workflows create --authorize mfprj-fz6nmearex563gd6-0-wtag
when deploying this flow to Argo Workflows for the first time.
See "Organizing Results" at https://docs.metaflow.org/ for more information about production tokens.

Bootstrapping vir

In [28]:
! python ../flows/cloud/event_triggered_linear_regression.py --environment=conda --production --branch champion --production argo-workflows trigger

Metaflow 2.10.6+ob(v1) executing TaxiFarePrediction for user:sandbox
Project: champion, Branch: prod.champion
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Workflow champion.prod.champion.taxifareprediction triggered on Argo Workflows (run-id argo-champion.prod.champion.taxifareprediction-xcmn2).
See the run in the UI at https://ui-pw-323314244.outerbounds.dev/TaxiFarePrediction/argo-champion.prod.champion.taxifareprediction-xcmn2


## Task 2: Build the challenger
Develop a second model, by using the same `TaxiFarePrediction` architecture. Then, deploy the flow to Argo as the `--branch challenger`. 
<br>
<br>
Hint: Modify the `linear_model` step. 
<br>
Bonus: Write a paragraph summary of how you developed the second model and tested it before deploying the challenger flow. Let us know in Slack what you found challenging about the task? 

In [11]:
%%writefile ../flows/cloud/challenger.py
from metaflow import FlowSpec, step, card, conda_base, current, Parameter, Flow, trigger, project
from metaflow.cards import Markdown, Table, Image, Artifact

URL = "https://outerbounds-datasets.s3.us-west-2.amazonaws.com/taxi/latest.parquet"
DATETIME_FORMAT = "%Y-%m-%d %H:%M:%S"

@project(name="challenger")
@trigger(events=["s3"])
@conda_base(
    libraries={
        "pandas": "1.4.2",
        "pyarrow": "11.0.0",
        "numpy": "1.22.4",
        "scikit-learn": "1.1.2",
        "xgboost": "1.7.6"
    }
)
class TaxiFarePrediction(FlowSpec):
    data_url = Parameter("data_url", default=URL)

    def transform_features(self, df):
        # TODO:
        obviously_bad_data_filters = [
                df.fare_amount > 0,  # fare_amount in US Dollars
                df.trip_distance <= 100,  # trip_distance in miles
                df.trip_distance > 0
             ]
        for f in obviously_bad_data_filters:
            df = df[f]
        # Try to complete tasks 2 and 3 with this function doing nothing like it currently is.
        # Understand what is happening.
        # Revisit task 1 and think about what might go in this function.

        return df

    @step
    def start(self):
        import pandas as pd
        from sklearn.model_selection import train_test_split

        self.df = self.transform_features(pd.read_parquet(self.data_url))
        # NOTE: we are split into training and validation set in the validation step which uses cross_val_score.
        # This is a simple/naive way to do this, and is meant to keep this example simple, to focus learning on deploying Metaflow flows.
        # In practice, you want split time series data in more sophisticated ways and run backtests.
        self.X = self.df["trip_distance"].values.reshape(-1, 1)
        self.y = self.df["total_amount"].values
        # self.y = self.df["fare_amount"].values
        self.next(self.xgb)

    @step
    def xgb(self):
        "Fit a single variable, linear model to the data."
        from xgboost import XGBRegressor

        # TODO: Play around with the model if you are feeling it.
        self.model = XGBRegressor()
        self.model.fit(self.X,self.y)
        
        self.next(self.validate)

    def gather_sibling_flow_run_results(self):
        # storage to populate and feed to a Table in a Metaflow card
        rows = []

        # loop through runs of this flow
        for run in Flow(self.__class__.__name__):
            if run.id != current.run_id:
                if run.successful:
                    icon = "✅"
                    msg = "OK"
                    score = str(run.data.scores.mean())
                else:
                    icon = "❌"
                    msg = "Error"
                    score = "NA"
                    for step in run:
                        for task in step:
                            if not task.successful:
                                msg = task.stderr
                row = [
                    Markdown(icon),
                    Artifact(run.id),
                    Artifact(run.created_at.strftime(DATETIME_FORMAT)),
                    Artifact(score),
                    Markdown(msg),
                ]
                rows.append(row)
            else:
                rows.append(
                    [
                        Markdown("✅"),
                        Artifact(run.id),
                        Artifact(run.created_at.strftime(DATETIME_FORMAT)),
                        Artifact(str(self.scores.mean())),
                        Markdown("This run..."),
                    ]
                )
        return rows

    @card(type="corise")
    @step
    def validate(self):
        from sklearn.model_selection import cross_val_score

        self.scores = cross_val_score(self.model, self.X, self.y, cv=5)
        current.card.append(Markdown("# Taxi Fare Prediction Results"))
        current.card.append(
            Table(
                self.gather_sibling_flow_run_results(),
                headers=["Pass/fail", "Run ID", "Created At", "R^2 score", "Stderr"],
            )
        )
        self.next(self.end)

    @step
    def end(self):
        print("Success!")


if __name__ == "__main__":
    TaxiFarePrediction()

Overwriting ../flows/cloud/challenger.py


In [12]:
! python ../flows/cloud/challenger.py --environment=conda --production --branch challenger --production argo-workflows create

Metaflow 2.10.6+ob(v1) executing TaxiFarePrediction for user:sandbox
Project: challenger, Branch: prod.challenger
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Deploying challenger.prod.challenger.taxifareprediction to Argo Workflows...
It seems this is the first time you are deploying challenger.prod.challenger.taxifareprediction to Argo Workflows.

A new production token generated.

The namespace of this production flow is
    production:mfprj-cdpm3wg3xft7bpmn-0-iqnt
To analyze results of this production flow add this line in your notebooks:
    namespace("production:mfprj-cdpm3wg3xft7bpmn-0-iqnt")
If you want to authorize other people to deploy new versions of this flow to Argo Workflows, they need to call
    argo-workflows create --authorize mfprj-cdpm3wg3xft7bpmn-0-iqnt
when deploying this flow to Argo Workflows for the first time.
See "Organizing Results" at https://docs.metaflow.org/ for more information about production tokens.

Boots

In [27]:
! python ../flows/cloud/challenger.py --environment=conda --production --branch challenger --production argo-workflows trigger

Metaflow 2.10.6+ob(v1) executing TaxiFarePrediction for user:sandbox
Project: challenger, Branch: prod.challenger
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Workflow challenger.prod.challenger.taxifareprediction triggered on Argo Workflows (run-id argo-challenger.prod.challenger.taxifareprediction-7lgq8).
See the run in the UI at https://ui-pw-323314244.outerbounds.dev/TaxiFarePrediction/argo-challenger.prod.challenger.taxifareprediction-7lgq8


## Task 3: Analyze the results
Return to this notebook, and read in the results of the challenger and champion flow using the Metaflow Client API.
<br><br>

#### Questions
- Does your model perform better on the metrics you selected? 
- Think about your day job, how would you go about assessing whether to roll forward the production "champion" to your new model? 
    - What gives you confidence one model is better than another?
    - What kinds of information do you need to monitor to get buy-in from stakeholders that model A is preferable to model B?  

In [26]:
from metaflow import Flow, namespace

# these values are unique to your deployment!
CHAMPION_MODEL_NAMESPACE = "production:mfprj-fz6nmearex563gd6-0-wtag" 
CHALLENGER_MODEL_NAMESPACE = "production:mfprj-cdpm3wg3xft7bpmn-0-iqnt" 

best_score = -1; winner = None; winner_namespace = None
for n,d in zip([CHAMPION_MODEL_NAMESPACE, CHALLENGER_MODEL_NAMESPACE],
                ['LinearRegression','XGBoost']):
    namespace(n)
    run = Flow('TaxiFarePrediction').latest_successful_run
    cross_val_score = run.data.scores.mean()
    print(d,': cross_val_score =',cross_val_score)
    if cross_val_score > best_score:
        best_score = cross_val_score
        winner = d
        winner_namespace = n
print(f'The winner is the {winner} model, with cross validation score of {best_score}.')

LinearRegression : cross_val_score = 0.8956057817631933
XGBoost : cross_val_score = 0.9047663921479548
The winner is the XGBoost model, with cross validation score of 0.9047663921479548.


## CONGRATULATIONS! 🎉✨🍾
If you made it this far, you have completed the Full Stack Machine Learning Corise course. 
We are so glad that you chose to learn with us, and hope to see you again in future courses. Stay tuned for more content and come join us in [Slack](http://slack.outerbounds.co/) to keep learning about Metaflow!